# Test Notebook

In this notebook, we work through some functionalities, showing the validity of the implementation. Specifically, we show the STLC with the following extensions:
- Binary operations (arithmetic and logical)
- If then else
- Conjunctive and disjunctive types (product and sum types)

## Binary operations

### Arithmetic operations

In [2]:
from files import *
# Binary Operation: 42 + 10
x = (42)
y = (10)
add_expr = BinOp(x, '+', y)  # 42 + 10

# Type-check and evaluate
print("Type-checking 42 + 10...")

add_type = type_check(add_expr, TypeContext())
print(f"Type: {add_type}")  # Expected: Int
add_result = eval_expr(add_expr, Env())
print(f"Result: {add_result}")  # Expected: (52)

Type-checking 42 + 10...
Type: Int
Result: 52


In [3]:
# Binary Operation: 42 * 2
mul_expr = BinOp(IntValue(42), '*', IntValue(2))  # 42 * 2

# Type-check and evaluate
print("Type-checking 42 * 2...")

mul_type = type_check(mul_expr, TypeContext())
print(f"Type: {mul_type}")  # Expected: Int
mul_result = eval_expr(mul_expr, Env())
print(f"Result: {mul_result}")  # Expected: (84)

Type-checking 42 * 2...
Type: Int
Result: 84


### Logical operations

In [4]:
# Logical Operation: true && false
bool_expr = BinOp((True), '&&', BoolValue(False))  # true && false

# Type-check and evaluate
print("Type-checking true && false...")
bool_type = type_check(bool_expr, TypeContext())
print(f"Type: {bool_type}")  # Expected: Bool
bool_result = eval_expr(bool_expr, Env())
print(f"Result: {bool_result}")  # Expected: (False)

Type-checking true && false...
Type: Bool
Result: false


In [5]:
# Logical Operation: true || false
or_expr = BinOp(BoolValue(True), '||', BoolValue(False))  # true || false

# Type-check and evaluate
print("Type-checking true || false...")
or_type = type_check(or_expr, TypeContext())
print(f"Type: {or_type}")  # Expected: Bool
or_result = eval_expr(or_expr, Env())
print(f"Result: {or_result}")  # Expected: (True)

Type-checking true || false...
Type: Bool
Result: true


## Function Application

In [6]:
# Define a lambda function: λx: Int. x + 1
x_var = Var('x')
lambda_expr = Abs('x', IntType(), BinOp(x_var, '+', IntValue(1)))  # λx: Int. x + 1

# Apply the function to the value 5
application = App(lambda_expr, IntValue(5))  # (λx. x + 1)(5)

# Type-check and evaluate
print("Type-checking (λx: Int. x + 1)(5)...")
app_type = type_check(application, TypeContext())
print(f"Type: {app_type}")  # Expected: Int
app_result = eval_expr(application, Env())
print(f"Result: {app_result}")  # Expected: (6)

Type-checking (λx: Int. x + 1)(5)...
Type: Int
Result: 6


### If statement

In [7]:
# Define a lambda function: λx: Int. if x > 5 then true else false
cond_expr = If(BinOp(Var('x'), '>', IntValue(5)), BoolValue(True), BoolValue(False))  # if x > 5 then true else false
lambda_cond_expr = Abs('x', IntType(), cond_expr)  # λx: Int. if x > 5 then true else false

# Apply the function to the value 7
application_cond = App(lambda_cond_expr, IntValue(7))  # (λx: Int. if x > 5 then true else false)(7)

# Type-check and evaluate
print("Type-checking (λx: Int. if x > 5 then true else false)(7)...")

cond_type = type_check(application_cond, TypeContext())
print(f"Type: {cond_type}")  # Expected: Bool
cond_result = eval_expr(application_cond, Env())
print(f"Result: {cond_result}")  # Expected: (True)

Type-checking (λx: Int. if x > 5 then true else false)(7)...
Type: Bool
Result: true


### Conjunctive types

In [16]:
# Define types for testing
bool_type = BoolType()
int_type = IntType()

# Create a pair (true, 42) of type (Bool ∧ Int)
pair_expr = Pair(BoolValue(True), IntValue(42))

# Type-check the pair expression
context = TypeContext()
try:
    pair_type = type_check(pair_expr, context)
    print(f"Type of the pair: {pair_type}")  # Expected: (Bool ∧ Int)
except TypeError as e:
    print(f"Type error: {e}")

# Test extracting the first component using fst
fst_expr = Fst(pair_expr)

fst_type = type_check(fst_expr, context)
print(f"Type of fst(pair): {fst_type}")  # Expected: Bool
print(f"Result of fst(pair): {eval_expr(fst_expr, Env())}")  # Expected: (True)


# Test extracting the second component using snd
snd_expr = Snd(pair_expr)

snd_type = type_check(snd_expr, context)
print(f"Type of snd(pair): {snd_type}")  # Expected: Int
print(f"Result of snd(pair): {eval_expr(snd_expr, Env())}")  # Expected: (42)

Type of the pair: (Bool ∧ Int)
Type of fst(pair): Bool
Result of fst(pair): true
Type of snd(pair): Int
Result of snd(pair): 42


### Disjunctive types

In [8]:
# Disjunction: Int ∨ Bool
or_type = OrType(IntType(), BoolType())

x = Var('x')
context = TypeContext()
context.add('x', IntType())
env = Env()
env.extend('x', IntValue(42))

# Create inl(42) and inr(true)
inl_expr = Inl(x, or_type)  # Left injection (Int)
inr_expr = Inr(BoolValue(True), or_type)  # Right injection (Bool)

# Define the case expression
# case inl_expr of inl(x) => x + 1 | inr(y) => 0
left_case = BinOp(x, '+', IntValue(1))  # x + 1
right_case = IntValue(0)  # Just return 0
case_expr = Case(inl_expr, left_case, right_case)



# Type-check and evaluate the case expression
print("Type-checking case expression with inl(42)...")
case_type = type_check(case_expr, context)  # Expected: Int
case_result = eval_expr(case_expr, env)
print(f"Type: {case_type}")
print(f"Result: {case_result}")  # Expected: (43)

Type-checking case expression with inl(42)...
Type: Int
Result: 43


# Proving propositional logic

#### Proving (A ∧ B) → A

In [9]:
from files import *

# Define the types A and B
A = IntType()
B = BoolType()

# The theorem we are proving: (A ∧ B) → A
theorem_type = FuncType(AndType(A, B), A)

# Define the proof: λp. fst(p)
p = Var('p')  # p is the pair (A ∧ B)
proof_term = Abs('p', AndType(A, B), Fst(p))  # λp: (A ∧ B). fst(p)

# Type-check the proof
context = TypeContext()
result_type = type_check(proof_term, context)
print(f"Proof type: {result_type}")  # Expected: (A ∧ B) → A

Proof type: ((Int ∧ Bool) -> Int)


In [12]:
# The theorem we are proving: A → (A ∨ B)
theorem_type = FuncType(A, OrType(A, B))

# Define the proof: λa. inl(a)
a = Var('a')
inl_exp = Inl(a, OrType(A, B))
proof_term = Abs('a', A, inl_exp)  # λa: A. inl(a)

# Type-check the proof
result_type = type_check(proof_term, context)
print(f"Proof type: {result_type}")  # Expected: A → (A ∨ B)

Proof type: (Int -> (Int ∨ Bool))


In [13]:
# Define types A, B, and C
A = IntType()
B = BoolType()
C = IntType()

# The theorem we are proving: (A → C) ∧ (B → C) → (A ∨ B → C)
theorem_type = FuncType(
    AndType(FuncType(A, C), FuncType(B, C)),  # (A → C) ∧ (B → C)
    FuncType(OrType(A, B), C)                 # (A ∨ B) → C
)

# Define the proof: λf. λx. case x of (a => fst(f)(a), b => snd(f)(b))
f = Var('f')
x = Var('x')
left_case = App(Fst(f), Var('a'))  # Apply fst(f) to a
right_case = App(Snd(f), Var('b'))  # Apply snd(f) to b
case_expr = Case(x, left_case, right_case)  # case x of (a => fst(f)(a), b => snd(f)(b))
proof_term = Abs('f', AndType(FuncType(A, C), FuncType(B, C)),
                 Abs('x', OrType(A, B), case_expr))  # λf. λx. case x of ...

# Type-check the proof

result_type = type_check(proof_term, context)
print(f"Proof type: {result_type}")  # Expected: ((A → C) ∧ (B → C)) → (A ∨ B) → C

Proof type: (((Int -> Int) ∧ (Bool -> Int)) -> ((Int ∨ Bool) -> Int))
